In [262]:
import pandas as pd
import datetime as dt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

weekly = pd.read_csv('../Dataset/Boulder_Weekly.csv')
weekly['Date'] = pd.to_datetime(weekly['Date'])
weekly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           239 non-null    datetime64[ns]
 1   Energy__kWh_   239 non-null    float64       
 2   Day            239 non-null    int64         
 3   Minimum T      239 non-null    int64         
 4   Maximum T      239 non-null    int64         
 5   Snow           239 non-null    float64       
 6   Precipitation  239 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(3)
memory usage: 13.2 KB


In [263]:
if weekly['Date'].is_monotonic_increasing == False:
    weekly.sort_values(by='Date', ascending=True, inplace=True)

weekly['Month'] = weekly['Date'].dt.month_name()
weekly.drop(columns={'Date', 'Day'}, inplace=True)

    Scale the Dataset with MinMaxScaler / One-Hot Encode and Extract the Entire Scaled Dataset

In [264]:
# Define the columns we need to scale
columns_to_scale = ['Energy__kWh_', 'Minimum T', 'Maximum T', 'Snow', 'Precipitation']

# MinMax scaling for numerical columns
scaler = MinMaxScaler()
weekly_scaled = weekly.copy()
weekly_scaled[columns_to_scale] = scaler.fit_transform(weekly[columns_to_scale])

# Define the columns we need to use for One-Hot Encoding
categorical_columns = ['Month']

# One-hot encoding for 'Weekday' and 'Month'
onehot_encoder = OneHotEncoder(sparse=False)
categorical_encoded = onehot_encoder.fit_transform(weekly[categorical_columns])

# Get the feature names from the encoder
encoded_columns = []
for col, values in zip(categorical_columns, onehot_encoder.categories_):
    encoded_columns.extend([f'{col}_{value}' for value in values])

# Create DataFrame with encoded columns
categorical_encoded_df = pd.DataFrame(categorical_encoded, columns=encoded_columns)

# Concatenate the new encoded columns to the original DataFrame
weekly_scaled = pd.concat([weekly_scaled, categorical_encoded_df], axis=1)

# Drop the original categorical columns
weekly_scaled = weekly_scaled.drop(categorical_columns, axis=1)
weekly_scaled.columns

Index(['Energy__kWh_', 'Minimum T', 'Maximum T', 'Snow', 'Precipitation',
       'Month_April', 'Month_August', 'Month_December', 'Month_February',
       'Month_January', 'Month_July', 'Month_June', 'Month_March', 'Month_May',
       'Month_November', 'Month_October', 'Month_September'],
      dtype='object')

    Divided the dataset into training, testing, and validation datasets according to 0.70, 0.20, and 0.10, respectively.

In [265]:
# Split the dataset into training, validation, and testing sets
def split_dataset(df, train_ratio, val_ratio):

    total_size = len(df)
    train_size = int(train_ratio * total_size)
    val_size = int(val_ratio * total_size)
    test_size = total_size - train_size - val_size

    train_df = df[:train_size]
    val_df = df[train_size:train_size + val_size]
    test_df = df[train_size + val_size:]

    assert len(train_df) + len(val_df) + len(test_df) == total_size, "Dataset not split correctly."

    print(f'Training split ratio:   {round(len(train_df) / len(df), 3)}')
    print(f'Validation split ratio: {round(len(val_df) / len(df), 3)}')
    print(f'Testing split ratio:    {round(len(test_df) / len(df), 3)}')
    print("\nShapes of the datasets:")
    print(train_df.shape, val_df.shape, test_df.shape)

    return train_df, val_df, test_df

In [266]:
train_weekly_scaled, val_weekly_scaled, test_weekly_scaled = split_dataset(weekly_scaled, train_ratio=0.7, val_ratio=0.2)

Training split ratio:   0.699
Validation split ratio: 0.197
Testing split ratio:    0.105

Shapes of the datasets:
(167, 17) (47, 17) (25, 17)


    Create sequences for the Transformer model

In [267]:
# Reshape the data
def create_sequences(data, sequence_length):
    inputs = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequence = data.iloc[i:i + sequence_length].values
        target = data.iloc[i + sequence_length]['Energy__kWh_']  # Predict the next value
        inputs.append(sequence)
        targets.append(target)

    inputs_array = np.array(inputs)
    targets_array = np.array(targets)
    
    print(f'Dataset split into sequences:')
    print(f'Sequences shape: {inputs_array.shape}')
    print(f'Targets shape: {targets_array.shape}\n')

    return np.array(inputs), np.array(targets)

In [268]:
sequence_length = 22
num_features = len(weekly_scaled.columns)

# Create the training, validation, and test data sequences
train_data_inputs, train_data_targets = create_sequences(train_weekly_scaled, sequence_length)
val_data_inputs, val_data_targets = create_sequences(val_weekly_scaled, sequence_length)
test_data_inputs, test_data_targets = create_sequences(test_weekly_scaled, sequence_length)

Dataset split into sequences:
Sequences shape: (145, 22, 17)
Targets shape: (145,)

Dataset split into sequences:
Sequences shape: (25, 22, 17)
Targets shape: (25,)

Dataset split into sequences:
Sequences shape: (3, 22, 17)
Targets shape: (3,)



In [269]:
# The input Datasets must have this input shape (-1, sequence_length, num_features)
train_data_inputs = train_data_inputs.reshape((-1, sequence_length, num_features))
val_data_inputs = val_data_inputs.reshape((-1, sequence_length, num_features))
test_data_inputs = test_data_inputs.reshape((-1, sequence_length, num_features))

train_data_inputs.shape, val_data_inputs.shape, test_data_inputs.shape

((145, 22, 17), (25, 22, 17), (3, 22, 17))

    Create the Transformer Models

In [270]:
%run "../Code/Transformer.ipynb"

# Define the hyperparameters of the manual model
input_shape = (sequence_length, num_features)
num_heads = 1
d_ff = 64
num_layers = 3
dropout_rate = 0.1
encoder_mask = None
decoder_mask = tf.linalg.band_part(tf.ones((sequence_length, sequence_length)), -1, 0)  # Create a lower triangular mask
decoder_mask = 1 - decoder_mask  # Invert the mask

# Create the transformer model
manul_model = TransformerModel(input_shape, num_heads, d_ff, num_layers, dropout_rate, encoder_mask, decoder_mask)

manul_model.summary()

Model: "model_20"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, 22, 17)]     0           []                               
                                                                                                  
 multi_head_attention_182 (Mult  (None, None, 17)    1224        ['input_21[0][0]',               
 iHeadAttention)                                                  'input_21[0][0]',               
                                                                  'input_21[0][0]']               
                                                                                                  
 dropout_366 (Dropout)          (None, None, 17)     0           ['multi_head_attention_182[0][0]'
                                                                 ]                         

In [271]:
# Create the transformer model
keras_model = keras_transformer_model(input_shape, num_heads, d_ff, num_layers, dropout_rate)

keras_model.summary()

Model: "model_21"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_22 (InputLayer)          [(None, 22, 17)]     0           []                               
                                                                                                  
 multi_head_attention_189 (Mult  (None, 22, 17)      1579        ['input_22[0][0]',               
 iHeadAttention)                                                  'input_22[0][0]']               
                                                                                                  
 dropout_381 (Dropout)          (None, 22, 17)       0           ['multi_head_attention_189[0][0]'
                                                                 ]                                
                                                                                           

    Compile the Models

In [272]:
def root_mean_squared_error(y_true, y_pred):
    return tf.keras.backend.sqrt(
        tf.keras.backend.mean(
            tf.keras.backend.square(
                y_pred - y_true
            )
        ) + 1e-9
    )

In [273]:
# Define the learning rate for Adam optimizer
learning_rate = 0.2

# Compile the manual model
manul_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss='mse',  metrics=['mae', 'mse', root_mean_squared_error])

# Compile the keras model
keras_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss='mse',  metrics=['mae', 'mse', root_mean_squared_error])

    Train the Models

In [274]:
# Define the parameters for training
epochs = 200
batch_size = 514

# Convert the data to float32
train_data_inputs = train_data_inputs.astype('float32')
train_data_targets = train_data_targets.astype('float32')

val_data_inputs = val_data_inputs.astype('float32')
val_data_targets = val_data_targets.astype('float32')

train_data_inputs.shape, train_data_targets.shape, val_data_inputs.shape, val_data_targets.shape

((145, 22, 17), (145,), (25, 22, 17), (25,))

In [275]:
# Train the manual model
manul_model.fit(train_data_inputs, train_data_targets,
          validation_data=(val_data_inputs, val_data_targets),
          epochs=epochs, batch_size=batch_size)

Epoch 1/200


1/1 [==============================] - 6s 6s/step - loss: 0.4450 - mae: 0.5350 - mse: 0.4450 - root_mean_squared_error: 0.6671 - val_loss: 27.2776 - val_mae: 5.2223 - val_mse: 27.2776 - val_root_mean_squared_error: 5.2228
Epoch 2/200
1/1 [==============================] - 0s 72ms/step - loss: 27.2221 - mae: 5.1944 - mse: 27.2221 - root_mean_squared_error: 5.2175 - val_loss: 0.0883 - val_mae: 0.2889 - val_mse: 0.0883 - val_root_mean_squared_error: 0.2972
Epoch 3/200
1/1 [==============================] - 0s 67ms/step - loss: 0.4150 - mae: 0.5385 - mse: 0.4150 - root_mean_squared_error: 0.6442 - val_loss: 15.2436 - val_mae: 3.9037 - val_mse: 15.2436 - val_root_mean_squared_error: 3.9043
Epoch 4/200
1/1 [==============================] - 0s 67ms/step - loss: 11.7976 - mae: 3.4158 - mse: 11.7976 - root_mean_squared_error: 3.4348 - val_loss: 5.8940 - val_mae: 2.4268 - val_mse: 5.8940 - val_root_mean_squared_error: 2.4278
Epoch 5/200
1/1 [==============================] - 0s 63ms/step - loss

In [ ]:
# Train the keras model
keras_model.fit(train_data_inputs, train_data_targets,
          validation_data=(val_data_inputs, val_data_targets),
          epochs=epochs, batch_size=batch_size)

Epoch 1/200
1/1 [==============================] - 12s 12s/step - loss: 2.5907 - mae: 1.4821 - mse: 2.5907 - root_mean_squared_error: 1.6096 - val_loss: 29.7330 - val_mae: 5.4524 - val_mse: 29.7330 - val_root_mean_squared_error: 5.4528
Epoch 2/200
1/1 [==============================] - 0s 145ms/step - loss: 30.1648 - mae: 5.4798 - mse: 30.1648 - root_mean_squared_error: 5.4923 - val_loss: 6.4220 - val_mae: 2.5333 - val_mse: 6.4220 - val_root_mean_squared_error: 2.5342
Epoch 3/200
1/1 [==============================] - 0s 137ms/step - loss: 7.1283 - mae: 2.6417 - mse: 7.1283 - root_mean_squared_error: 2.6699 - val_loss: 1.6185 - val_mae: 1.2704 - val_mse: 1.6185 - val_root_mean_squared_error: 1.2722
Epoch 4/200
1/1 [==============================] - 0s 138ms/step - loss: 1.0019 - mae: 0.9765 - mse: 1.0019 - root_mean_squared_error: 1.0010 - val_loss: 5.9899 - val_mae: 2.4465 - val_mse: 5.9899 - val_root_mean_squared_error: 2.4474
Epoch 5/200
1/1 [==============================] - 0s 139

    Evaluate the Models

In [ ]:
# Evaluate on validation set
val_metrics_manul = manul_model.evaluate(val_data_inputs, val_data_targets, return_dict=True)
val_metrics_keras = keras_model.evaluate(val_data_inputs, val_data_targets, return_dict=True)

# Evaluate on test set
test_metrics_manul = manul_model.evaluate(test_data_inputs, test_data_targets, return_dict=True)
test_metrics_keras = keras_model.evaluate(test_data_inputs, test_data_targets, return_dict=True)

# Extract individual metrics
val_loss_manul, val_mae_manul, val_mse_manul, val_rmse_manul = val_metrics_manul['loss'], val_metrics_manul['mae'], val_metrics_manul['mse'], val_metrics_manul['root_mean_squared_error']
test_loss_manul, test_mae_manul, test_mse_manul, test_rmse_manul = test_metrics_manul['loss'], test_metrics_manul['mae'], test_metrics_manul['mse'], test_metrics_manul['root_mean_squared_error']

val_loss_keras, val_mae_keras, val_mse_keras, val_rmse_keras = val_metrics_keras['loss'], val_metrics_keras['mae'], val_metrics_keras['mse'], val_metrics_keras['root_mean_squared_error']
test_loss_keras, test_mae_keras, test_mse_keras, test_rmse_keras = test_metrics_keras['loss'], test_metrics_keras['mae'], test_metrics_keras['mse'], test_metrics_keras['root_mean_squared_error']

print('\n\nManual Transformer:\n-------------------')
print(f'Validation Loss: {val_loss_manul}, Validation MSE: {val_mse_manul}, Validation MAE: {val_mae_manul}, Validation RMSE: {val_rmse_manul}')
print(f'Test Loss: {test_loss_manul}, Test MSE: {test_mse_manul}, Test MAE: {test_mae_manul}, Test RMSE: {test_rmse_manul}')

print('\nKeras Transformer:\n------------------')
print(f'Validation Loss: {val_loss_keras}, Validation MSE: {val_mse_keras}, Validation MAE: {val_mae_keras}, Validation RMSE: {val_rmse_keras}')
print(f'Test Loss: {test_loss_keras}, Test MSE: {test_mse_keras}, Test MAE: {test_mae_keras}, Test RMSE: {test_rmse_keras}')


1/1 [==============================] - 0s 38ms/step - loss: 0.3863 - mae: 0.6206 - mse: 0.3863 - root_mean_squared_error: 0.6215


Manual Transformer:
-------------------
Validation Loss: 0.06344471871852875, Validation MSE: 0.06344471871852875, Validation MAE: 0.2426632195711136, Validation RMSE: 0.2518823742866516
Test Loss: 0.35656213760375977, Test MSE: 0.35656213760375977, Test MAE: 0.5961843729019165, Test RMSE: 0.5971282720565796

Keras Transformer:
------------------
Validation Loss: 0.07636135816574097, Validation MSE: 0.07636135816574097, Validation MAE: 0.2679590582847595, Validation RMSE: 0.2763355076313019
Test Loss: 0.38632282614707947, Test MSE: 0.3863227963447571, Test MAE: 0.6206419467926025, Test RMSE: 0.6215487122535706


In [ ]:
# Assuming manul_model.predict returns the predictions
val_predictions_manul = manul_model.predict(val_data_inputs)
test_predictions_manul = manul_model.predict(test_data_inputs)

# Assuming keras_model.predict returns the predictions
val_predictions_keras = keras_model.predict(val_data_inputs)
test_predictions_keras  = keras_model.predict(test_data_inputs)

# Calculate MAE and RMSE for validation set
val_mae_manul = np.mean(np.abs(val_data_targets - val_predictions_manul))
val_rmse_manul = np.sqrt(np.mean(np.square(val_data_targets - val_predictions_manul)))

val_mae_keras  = np.mean(np.abs(val_data_targets - val_predictions_keras ))
val_rmse_keras  = np.sqrt(np.mean(np.square(val_data_targets - val_predictions_keras )))

# Calculate MAE and RMSE for test set
test_mae_manul = np.mean(np.abs(test_data_targets - test_predictions_manul))
test_rmse_manul = np.sqrt(np.mean(np.square(test_data_targets - test_predictions_manul)))

test_mae_keras  = np.mean(np.abs(test_data_targets - test_predictions_keras ))
test_rmse_keras  = np.sqrt(np.mean(np.square(test_data_targets - test_predictions_keras )))


print('\n\nManual Transformer:\n-------------------')
print(f'Validation MAE: {val_mae_manul}')
print(f'Validation RMSE: {val_rmse_manul}')
print(f'\nTest MAE: {test_mae_manul}')
print(f'Test RMSE: {test_rmse_manul}')
print('\n==============================')
print('\nKeras Transformer:\n------------------')
print(f'Validation MAE: {val_mae_keras }')
print(f'Validation RMSE: {val_rmse_keras }')
print(f'\nTest MAE: {test_mae_keras }')
print(f'Test RMSE: {test_rmse_keras }')

1/1 [==============================] - 0s 34ms/step


Manual Transformer:
-------------------
Validation MAE: 0.2426632046699524
Validation RMSE: 0.2518823444843292

Test MAE: 0.5961843654245372
Test RMSE: 0.5971282407581258


Keras Transformer:
------------------
Validation MAE: 0.2679591178894043
Validation RMSE: 0.27633556723594666

Test MAE: 0.6206419799547538
Test RMSE: 0.6215487148291851


Manual Transformer:
-------------------
Validation MAE: 0.23751895129680634
Validation RMSE: 0.24693451821804047

Test MAE: 0.5918878594314744
Test RMSE: 0.5928387148608047

==============================

Keras Transformer:
------------------
Validation MAE: 0.25300586223602295
Validation RMSE: 0.261861115694046

Test MAE: 0.6056900992721553
Test RMSE: 0.6066191836564087